# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [1]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [2]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

L,U = diy_lu(a)

np.linalg.matrix_rank(a)

6

In [3]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)


In [4]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(a-L@u)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  2.220e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -4.528e-16  0.000e+00  6.939e-18  8.080e-04  1.902e-03]
 [ 0.000e+00  4.123e-16  0.000e+00 -1.634e-17  0.000e+00 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -2.220e-16  1.110e-16  1.665e-16]
 [ 0.000e+00  0.000e+00 -2.220e-16  5.551e-17  1.665e-16  1.665e-16]
 [ 0.000e+00  0.000e+00  1.110e-16  1.665e-16 -1.665e-16 -5.551e-17]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [5]:
a1 = a.copy()
a1[1, 1] = 3
print(a1)

[[3.    3.    3.    3.    3.    3.   ]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]]


Resulting matix still has full rank, but the naive LU routine breaks down.

In [6]:
np.linalg.matrix_rank(a1)

6

In [10]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


C:\Users\polina\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
C:\Users\polina\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [8]:
def minor(c):
    i=0
    flag = True
    while i!=N and flag == True:
        b = np.delete(c.copy(), i, 1)
        b = np.delete(b, i, 0)
        if np.linalg.det(b)==0:
            flag=False
        i+=1
    return flag

print(minor(a), minor(a1))


True True


### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [9]:
np.set_printoptions(precision=3)

def diy_lu_mod(a):
    N = a.shape[0]
    u = a.copy()
    L = np.eye(N)
    P = np.eye(N)
    for j in range(N-1):

        P_j = np.eye(N)
        m = np.argmax(abs(u[j+1:, j]))
    
        if u[m+j+1, j]!=u[j, j]:    
            P_j[m+j+1], P_j[j] = P_j[j].copy(), P_j[m+j+1].copy()
            
        u = P_j @ u
    
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        
        lam[j+1:, j] = -gamma
        u = lam  @ u
        lam[j+1:, j] = gamma
        L = L @ P_j @ lam 
        
        P = P @ P_j
    return P, P.T @ L, u
#Tests:
P,L,u=diy_lu_mod(a)
print('Matrix a:')
print('L matrix', L,'U matrix', u,'a - P @ L @ u', a - P @ L @ u,'Max deviation is ', np.max(abs(a - P @ L @ u)),  sep='\n')
print('Matrix a1:')
P1,L1,u1=diy_lu_mod(a1)
print('L matrix', L1,'U matrix', u1,'a1 - P @ L @ u', a1-P1@L1@u1,'Max deviation is ',np.max(abs(a1-P1@L1@u1)),  sep='\n')

Matrix a:
L matrix
[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    0.5   1.    0.    0.    0.   ]
 [1.    0.727 0.706 1.    0.    0.   ]
 [1.    0.857 0.41  0.835 1.    0.   ]
 [1.    0.941 0.178 0.426 0.789 1.   ]]
U matrix
[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -3.506e-01 -5.786e-01 -7.330e-01 -8.438e-01]
 [ 0.000e+00  0.000e+00  2.776e-17  2.421e-02  4.866e-02  6.961e-02]
 [ 0.000e+00  2.220e-16 -2.317e-17  0.000e+00 -6.462e-04 -1.516e-03]
 [ 0.000e+00 -1.751e-16  6.463e-18  0.000e+00  0.000e+00  6.730e-06]]
a - P @ L @ u
[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -2.220e-16  1.110e-16  1.665e-16]
 [ 0.000e+00  0.000e+00 -2.220e-16  5.551e-17  1.665e-16  1.665e-16]
 [ 0.000e+00  0.000e+00  1.110e-

Concusion: Maximum deviations are nearly to the machine zero, so we can trust modified LU decomposition